In [5]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [6]:
housing = pd.read_csv('Housing.csv')
housing

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [7]:
#Choosing the values 
num_vars = ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking']
df_new = housing[num_vars]
df_new

,price,area,bedrooms,bathrooms,stories,parking
0,13300000,7420,4,2,3,2
1,12250000,8960,4,4,4,3
2,12250000,9960,3,2,2,2
3,12215000,7500,4,2,2,3
4,11410000,7420,4,1,2,2
...,...,...,...,...,...,...
540,1820000,3000,2,1,1,2
541,1767150,2400,3,1,1,0
542,1750000,3620,2,1,1,0
543,1750000,2910,3,1,1,0


In [8]:
df_new.shape

(545, 6)

In [9]:
#Scaling the data
scaler = MinMaxScaler()
df_new[num_vars] = scaler.fit_transform(df_new[num_vars])
df_new.head()

,price,area,bedrooms,bathrooms,stories,parking
0,1.000000,0.396564,0.6,0.333333,0.666667,0.666667
1,0.909091,0.502405,0.6,1.000000,1.000000,1.000000
2,0.909091,0.571134,0.4,0.333333,0.333333,0.666667
3,0.906061,0.402062,0.6,0.333333,0.333333,1.000000
4,0.836364,0.396564,0.6,0.000000,0.333333,0.666667


In [22]:
X = df_new.iloc[:, 1:6].values
Y = df_new.iloc[:, 0].values

#Converting into tensors
X = torch.tensor(X)
Y = torch.tensor(Y)

#Defining the model
def model(X, W1, W2, W3, W4, W5, B):
    return W5*X[:,4] + W4*X[:,3] + W3*X[:,2] + W2*X[:,1] + W1*X[:,0] + B

#Defining the MSE
def loss_fn(Y_p, Y):
    squared_diffs = (Y_p - Y)**2
    return squared_diffs.mean()

#Defining the weights
W1 = torch.ones(())
W2 = torch.ones(())
W3 = torch.ones(())
W4 = torch.ones(())
W5 = torch.ones(())
B = torch.zeros(())

#test and train ratio as 20-80 
n_samples = X.shape[0]
n_val = int(0.2 * n_samples)


#Splitting the data into train and test 
train_X = X[train_indices]
train_Y = Y[train_indices]

val_X = X[val_indices]
val_Y = Y[val_indices]

In [23]:
train_X.size()

torch.Size([436, 5])

In [26]:
#Defining the training loop
def training_loop(n_epochs, optimizer, params, train_X, val_X, train_Y, val_Y):
    
    for epoch in range(1, n_epochs + 1):
        train_Y_p = model(train_X, *params) 
        train_loss = loss_fn(train_Y_p, train_Y)
                             
        val_Y_p = model(val_X, *params) 
        val_loss = loss_fn(val_Y_p, val_Y)
        
        optimizer.zero_grad()
        train_loss.backward() 
        optimizer.step()

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")
            
    return params

In [27]:
#Initializing the parameters 0.1
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-1
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.0120, Validation loss 0.0111
Epoch 1000, Training loss 0.0117, Validation loss 0.0108
Epoch 1500, Training loss 0.0117, Validation loss 0.0108
Epoch 2000, Training loss 0.0117, Validation loss 0.0108
Epoch 2500, Training loss 0.0117, Validation loss 0.0108
Epoch 3000, Training loss 0.0117, Validation loss 0.0108
Epoch 3500, Training loss 0.0117, Validation loss 0.0108
Epoch 4000, Training loss 0.0117, Validation loss 0.0108
Epoch 4500, Training loss 0.0117, Validation loss 0.0108
Epoch 5000, Training loss 0.0117, Validation loss 0.0108


tensor([0.4257, 0.0636, 0.3120, 0.1444, 0.1055, 0.0445], requires_grad=True)

In [28]:
#Initializing the parameters 0.01
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.0490, Validation loss 0.0527
Epoch 1000, Training loss 0.0224, Validation loss 0.0245
Epoch 1500, Training loss 0.0161, Validation loss 0.0171
Epoch 2000, Training loss 0.0141, Validation loss 0.0144
Epoch 2500, Training loss 0.0133, Validation loss 0.0131
Epoch 3000, Training loss 0.0128, Validation loss 0.0123
Epoch 3500, Training loss 0.0124, Validation loss 0.0118
Epoch 4000, Training loss 0.0122, Validation loss 0.0115
Epoch 4500, Training loss 0.0121, Validation loss 0.0113
Epoch 5000, Training loss 0.0120, Validation loss 0.0111


tensor([ 4.6381e-01,  1.8629e-01,  3.0316e-01,  1.1512e-01,  8.9013e-02,
        -3.7879e-04], requires_grad=True)

In [29]:
#Initializing the parameters 0.001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-3
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.2207, Validation loss 0.2168
Epoch 1000, Training loss 0.1366, Validation loss 0.1392
Epoch 1500, Training loss 0.1155, Validation loss 0.1193
Epoch 2000, Training loss 0.1007, Validation loss 0.1048
Epoch 2500, Training loss 0.0883, Validation loss 0.0924
Epoch 3000, Training loss 0.0778, Validation loss 0.0819
Epoch 3500, Training loss 0.0688, Validation loss 0.0728
Epoch 4000, Training loss 0.0611, Validation loss 0.0650
Epoch 4500, Training loss 0.0546, Validation loss 0.0584
Epoch 5000, Training loss 0.0489, Validation loss 0.0526


tensor([ 0.6957,  0.5682,  0.6638,  0.3377,  0.3738, -0.3913],
       requires_grad=True)

In [30]:
#Initializing the parameters 0.0001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-4
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.9812, Validation loss 0.9423
Epoch 1000, Training loss 0.7848, Validation loss 0.7536
Epoch 1500, Training loss 0.6350, Validation loss 0.6100
Epoch 2000, Training loss 0.5206, Validation loss 0.5007
Epoch 2500, Training loss 0.4331, Validation loss 0.4173
Epoch 3000, Training loss 0.3660, Validation loss 0.3537
Epoch 3500, Training loss 0.3146, Validation loss 0.3050
Epoch 4000, Training loss 0.2749, Validation loss 0.2676
Epoch 4500, Training loss 0.2443, Validation loss 0.2389
Epoch 5000, Training loss 0.2205, Validation loss 0.2167


tensor([ 0.8463,  0.7600,  0.9077,  0.7701,  0.7928, -0.5077],
       requires_grad=True)